# load dataset

In [8]:
import configparser
import logging
from ocs_sample_library_preview import *
import json
import pandas as pd
import os

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
#logger.setLevel(logging.DEBUG)

config = configparser.ConfigParser()
config.read('config.ini')

ocsClient = OCSClient(config.get('Access', 'ApiVersion'), config.get('Access', 'Tenant'), config.get('Access', 'Resource'), 
                        config.get('Credentials', 'ClientId'), config.get('Credentials', 'ClientSecret'))
        

namespace_id = config.get('Configurations', 'Namespace')

In [2]:
import sys
from pathlib import Path
import datetime

# before loading data, check if it has already been loaded. If the following returns values, data at least some of it is loaded and you may not need to run the following commands

In [3]:
ocsClient.acceptverbosity = True
for stream in ocsClient.Streams.getStreams(namespace_id,query='nasa.bearing*'):
    value = ocsClient.Streams.getLastValue(namespace_id,stream.Id)
    print(f'{stream.Id}: {value}')

NASA.Bearing1: None
NASA.Bearing4: None
NASA.Bearing2: None
NASA.Bearing3: None


In [4]:
data_dir = Path("C:/data/IMS/2nd_test_partII")
if not data_dir.is_dir():
    print("Cannot find data directory")

In [9]:

# function to bulk load data by bearing
def write_to_sds(dataset):
    # for each bearing 1 throuugh 4
    for bearing in range(1,5):
            try:
                # build a list of timestamp,index,bearing # channel data
                values = [{ "timestamp": key.isoformat(), "index": int(row[f'index']),"channel": row[f'Bearing {bearing}'] } for key,row in dataset.iterrows()]
                # write it!
                ocsClient.Streams.insertValues(namespace_id,f'Nasa.bearing{bearing}',json.dumps(values))
            except Exception as e:
                print(f'Error: {str(e)}')

# dataframe for loaded file
dataset = pd.DataFrame(columns=['Bearing 1','Bearing 2','Bearing 3','Bearing 4'])

# load datafiles
for filename in os.listdir(data_dir):
    #logging.debug(f'filename: {filename}')
    print(f'filename: {filename}')
    start_period = (datetime.datetime.strptime(filename,"%Y.%m.%d.%H.%M.%S")) 
    logging.debug(f'filename: {filename}, start_period: {start_period}')
    dataset=pd.read_csv(os.path.join(data_dir, filename), sep='\t', header=None, names=['Bearing 1','Bearing 2','Bearing 3','Bearing 4'])
    dataset['index'] = dataset.index
    dataset['index'] += 1
    dataset['timestamp'] = pd.date_range(start_period, periods=20480, freq='50us')
    dataset.set_index('timestamp',inplace=True)
    write_to_sds(dataset)

filename: 2004.02.12.10.52.39
filename: 2004.02.12.11.02.39
filename: 2004.02.12.11.12.39
filename: 2004.02.12.11.22.39
filename: 2004.02.12.11.32.39
filename: 2004.02.12.11.42.39
filename: 2004.02.12.11.52.39
filename: 2004.02.12.12.02.39
filename: 2004.02.12.12.12.39
filename: 2004.02.12.12.22.39
filename: 2004.02.12.12.32.39
filename: 2004.02.12.12.42.39
filename: 2004.02.12.12.52.39
filename: 2004.02.12.13.02.39
filename: 2004.02.12.13.12.39
filename: 2004.02.12.13.22.39
filename: 2004.02.12.13.32.39
filename: 2004.02.12.13.42.39
filename: 2004.02.12.13.52.39
filename: 2004.02.12.14.02.39
filename: 2004.02.12.14.12.39
filename: 2004.02.12.14.22.39
filename: 2004.02.12.14.32.39
filename: 2004.02.12.14.42.39
filename: 2004.02.12.14.52.39
filename: 2004.02.12.15.02.39
filename: 2004.02.12.15.12.39
filename: 2004.02.12.15.22.39
filename: 2004.02.12.15.32.39
filename: 2004.02.12.15.42.39
filename: 2004.02.12.15.52.39
filename: 2004.02.12.16.02.39
filename: 2004.02.12.16.12.39
filename: 

In [10]:
ocsClient.acceptverbosity = True
for stream in ocsClient.Streams.getStreams(namespace_id,query='nasa.bearing*'):
    value = ocsClient.Streams.getLastValue(namespace_id,stream.Id)
    print(f'{stream.Id}: {value}')

NASA.Bearing1: {'timestamp': '2004-02-19T06:22:40.02395Z', 'index': 20480.0, 'channel': -0.002}
NASA.Bearing4: {'timestamp': '2004-02-19T06:22:40.02395Z', 'index': 20480.0, 'channel': -0.002}
NASA.Bearing2: {'timestamp': '2004-02-19T06:22:40.02395Z', 'index': 20480.0, 'channel': 0.0}
NASA.Bearing3: {'timestamp': '2004-02-19T06:22:40.02395Z', 'index': 20480.0, 'channel': 0.0}
